In [1]:
import pandas as pd
from shapely.geometry import Point, Polygon
from shapely import geometry
import time

In [2]:
test = pd.read_csv('../datasets/clean/clean_bike_stations.csv')
test

,id,station_name,lat,long,status,area
0,1594046383808271024,Troy St & Jackson Blvd,41.877505,-87.704850,1,27
1,641,Central Park Ave & Bloomingdale Ave,41.914166,-87.716755,1,22
2,1683527931525155814,Public Rack - Cornell Ave & 87th Pl,41.736881,-87.583146,0,48
3,367,Racine Ave & 35th St,41.830689,-87.656211,0,60
4,1673852313397164648,Public Rack - Francisco Ave & Touhy Ave,42.011865,-87.701317,0,2
...,...,...,...,...,...,...
1411,20,Sheffield Ave & Kingsbury St,41.910522,-87.653106,0,8
1412,54,Ogden Ave & Chicago Ave,41.896362,-87.654061,0,24
1413,693,Baltimore Ave & 87th St,41.737336,-87.548515,1,46
1414,718,Ewing Ave & Burnham Greenway,41.712749,-87.534814,1,52


In [3]:
reference = pd.read_csv('../original_datasets/unclean_disadvantaged_areas.csv')
reference

,the_geom
0,MULTIPOLYGON (((-87.57366299996758 41.70796600...
1,MULTIPOLYGON (((-87.60151899995586 41.68632499...
2,MULTIPOLYGON (((-87.60036199996543 41.69284299...
3,MULTIPOLYGON (((-87.62032800000847 41.69942700...
4,MULTIPOLYGON (((-87.66036699995497 41.75771799...
...,...
275,MULTIPOLYGON (((-87.6624109999806 41.998169000...
276,MULTIPOLYGON (((-87.66572099996036 42.00497899...
277,MULTIPOLYGON (((-87.65500000001438 41.99831699...
278,MULTIPOLYGON (((-87.65029099995624 41.99662699...


In [4]:
reference.dtypes

the_geom    object
dtype: object

In [5]:
reference.loc[0].values

array(['MULTIPOLYGON (((-87.57366299996758 41.707966000031426, -87.57267600000398 41.707981000009006, -87.57259999999427 41.70798300000619, -87.57051000001076 41.70801200002498, -87.56971499996679 41.70802300001818, -87.56914899998023 41.70803099997312, -87.56872800000517 41.70803700000908, -87.56860900001972 41.708037999988655, -87.56825200004245 41.70804300000275, -87.56813400001899 41.70804600003349, -87.56789199996008 41.70804999997805, -87.56716800000734 41.70806099997388, -87.56692700002974 41.708065999986964, -87.56668499999031 41.70806899996619, -87.56596000002395 41.708080000016764, -87.56571899998117 41.708085000033755, -87.56547700000543 41.708088000007386, -87.56475200001057 41.70809899998144, -87.56451100000696 41.708102999973114, -87.56432800004386 41.70810899996785, -87.56390700003448 41.70811299997787, -87.56330300003867 41.7081209999717, -87.5631990000245 41.708123000005834, -87.56269900004557 41.708129999975974, -87.56209200003089 41.70813899999696, -87.5618310000122 

In [6]:
def convert_to_polygon(df):
    updated_polygons = []
    for row in df['the_geom']:
        target = row.replace('MULTIPOLYGON (((', '').replace(')))', '')
        points = target.split(', ')
        final = []
        for point in points:
            temp = point.split(' ')
            tup = float(temp[1].replace(')', '').replace('(', '')), float(temp[0].replace(')', '').replace('(', '')) 
            final.append(tup)
        polygons = Polygon(final)
        updated_polygons.append(polygons)
    return updated_polygons
    

In [7]:
unclean_areas = pd.read_csv('../original_datasets/unclean_areas.csv')
unclean_areas

,the_geom,PERIMETER,AREA,COMAREA_,COMAREA_ID,AREA_NUMBE,COMMUNITY,AREA_NUM_1,SHAPE_AREA,SHAPE_LEN
0,MULTIPOLYGON (((-87.60914087617894 41.84469250...,0,0,0,0,35,DOUGLAS,35,4.600462e+07,31027.054510
1,MULTIPOLYGON (((-87.59215283879394 41.81692934...,0,0,0,0,36,OAKLAND,36,1.691396e+07,19565.506153
2,MULTIPOLYGON (((-87.62879823733725 41.80189303...,0,0,0,0,37,FULLER PARK,37,1.991670e+07,25339.089750
3,MULTIPOLYGON (((-87.6067081256125 41.816813770...,0,0,0,0,38,GRAND BOULEVARD,38,4.849250e+07,28196.837157
4,MULTIPOLYGON (((-87.59215283879394 41.81692934...,0,0,0,0,39,KENWOOD,39,2.907174e+07,23325.167906
...,...,...,...,...,...,...,...,...,...,...
72,MULTIPOLYGON (((-87.69645961375822 41.70714491...,0,0,0,0,74,MOUNT GREENWOOD,74,7.558429e+07,48665.130539
73,MULTIPOLYGON (((-87.64215204651398 41.68508211...,0,0,0,0,75,MORGAN PARK,75,9.187734e+07,46396.419362
74,MULTIPOLYGON (((-87.83658087874365 41.98639611...,0,0,0,0,76,OHARE,76,3.718356e+08,173625.984660
75,MULTIPOLYGON (((-87.65455590025104 41.99816614...,0,0,0,0,77,EDGEWATER,77,4.844999e+07,31004.830946


In [8]:
unclean_areas = unclean_areas[['the_geom', 'AREA_NUMBE']]
unclean_areas

,the_geom,AREA_NUMBE
0,MULTIPOLYGON (((-87.60914087617894 41.84469250...,35
1,MULTIPOLYGON (((-87.59215283879394 41.81692934...,36
2,MULTIPOLYGON (((-87.62879823733725 41.80189303...,37
3,MULTIPOLYGON (((-87.6067081256125 41.816813770...,38
4,MULTIPOLYGON (((-87.59215283879394 41.81692934...,39
...,...,...
72,MULTIPOLYGON (((-87.69645961375822 41.70714491...,74
73,MULTIPOLYGON (((-87.64215204651398 41.68508211...,75
74,MULTIPOLYGON (((-87.83658087874365 41.98639611...,76
75,MULTIPOLYGON (((-87.65455590025104 41.99816614...,77


In [9]:
unclean_areas.sort_values(by='AREA_NUMBE')

,the_geom,AREA_NUMBE
9,MULTIPOLYGON (((-87.65455590025104 41.99816614...,1
19,MULTIPOLYGON (((-87.6846530946559 42.019484772...,2
30,MULTIPOLYGON (((-87.64102430213292 41.95480280...,3
5,MULTIPOLYGON (((-87.6744075678037 41.976103404...,4
47,MULTIPOLYGON (((-87.67336415409336 41.93234274...,5
...,...,...
71,MULTIPOLYGON (((-87.63373383514987 41.72885272...,73
72,MULTIPOLYGON (((-87.69645961375822 41.70714491...,74
73,MULTIPOLYGON (((-87.64215204651398 41.68508211...,75
74,MULTIPOLYGON (((-87.83658087874365 41.98639611...,76


In [10]:
clean_areas = convert_to_polygon(unclean_areas)
clean_areas

[<POLYGON ((41.845 -87.609, 41.845 -87.609, 41.845 -87.609, 41.845 -87.609, 4...>,
 <POLYGON ((41.817 -87.592, 41.817 -87.592, 41.817 -87.595, 41.817 -87.595, 4...>,
 <POLYGON ((41.802 -87.629, 41.802 -87.629, 41.802 -87.629, 41.801 -87.629, 4...>,
 <POLYGON ((41.817 -87.607, 41.817 -87.607, 41.816 -87.607, 41.816 -87.607, 4...>,
 <POLYGON ((41.817 -87.592, 41.817 -87.592, 41.817 -87.592, 41.817 -87.592, 4...>,
 <POLYGON ((41.976 -87.674, 41.976 -87.674, 41.976 -87.674, 41.975 -87.674, 4...>,
 <POLYGON ((41.786 -87.606, 41.786 -87.606, 41.786 -87.606, 41.786 -87.607, 4...>,
 <POLYGON ((41.803 -87.58, 41.802 -87.58, 41.802 -87.58, 41.802 -87.58, 41.80...>,
 <POLYGON ((41.786 -87.577, 41.786 -87.577, 41.786 -87.577, 41.786 -87.577, 4...>,
 <POLYGON ((41.998 -87.655, 41.998 -87.656, 41.998 -87.656, 41.998 -87.656, 4...>,
 <POLYGON ((41.968 -87.753, 41.968 -87.753, 41.968 -87.753, 41.968 -87.753, 4...>,
 <POLYGON ((42.005 -87.769, 42.005 -87.769, 42.005 -87.769, 42.005 -87.769, 4...>,
 <PO

In [11]:
clean_reference = convert_to_polygon(reference)
clean_reference

[<POLYGON ((41.708 -87.574, 41.708 -87.573, 41.708 -87.573, 41.708 -87.571, 4...>,
 <POLYGON ((41.686 -87.602, 41.686 -87.602, 41.686 -87.602, 41.686 -87.602, 4...>,
 <POLYGON ((41.693 -87.6, 41.693 -87.601, 41.693 -87.601, 41.693 -87.601, 41....>,
 <POLYGON ((41.699 -87.62, 41.699 -87.62, 41.699 -87.619, 41.699 -87.619, 41....>,
 <POLYGON ((41.758 -87.66, 41.758 -87.659, 41.758 -87.659, 41.758 -87.659, 41...>,
 <POLYGON ((41.716 -87.633, 41.717 -87.633, 41.717 -87.633, 41.718 -87.633, 4...>,
 <POLYGON ((41.698 -87.623, 41.697 -87.623, 41.697 -87.623, 41.697 -87.623, 4...>,
 <POLYGON ((41.707 -87.632, 41.707 -87.632, 41.707 -87.632, 41.707 -87.633, 4...>,
 <POLYGON ((41.699 -87.656, 41.699 -87.656, 41.699 -87.657, 41.699 -87.657, 4...>,
 <POLYGON ((41.689 -87.661, 41.689 -87.661, 41.689 -87.661, 41.689 -87.662, 4...>,
 <POLYGON ((41.691 -87.642, 41.691 -87.642, 41.692 -87.642, 41.692 -87.642, 4...>,
 <POLYGON ((41.785 -87.669, 41.785 -87.669, 41.783 -87.669, 41.783 -87.669, 4...>,
 <PO

In [12]:
def determine_within(df):
    statuses = []
    areas = []
    for i in range(len(df)):
        temp = geometry.Point(df.lat.loc[i], df.long.loc[i])
        status = 0
        for polygon in clean_reference:
            if (polygon.contains(temp)): 
                status = 1
        statuses.append(status)
        index = 0
        inside = False
        for polygon in clean_areas:
            if (polygon.contains(temp)): 
                areas.append(unclean_areas['AREA_NUMBE'].iloc[index])
                inside = True
            index += 1
        if inside == False:
            areas.append(0)
    df['status'] = statuses
    df['area'] = areas
    return df

In [13]:
test = determine_within(test)
test

,id,station_name,lat,long,status,area
0,1594046383808271024,Troy St & Jackson Blvd,41.877505,-87.704850,1,27
1,641,Central Park Ave & Bloomingdale Ave,41.914166,-87.716755,1,22
2,1683527931525155814,Public Rack - Cornell Ave & 87th Pl,41.736881,-87.583146,0,48
3,367,Racine Ave & 35th St,41.830689,-87.656211,0,60
4,1673852313397164648,Public Rack - Francisco Ave & Touhy Ave,42.011865,-87.701317,0,2
...,...,...,...,...,...,...
1411,20,Sheffield Ave & Kingsbury St,41.910522,-87.653106,0,8
1412,54,Ogden Ave & Chicago Ave,41.896362,-87.654061,0,24
1413,693,Baltimore Ave & 87th St,41.737336,-87.548515,1,46
1414,718,Ewing Ave & Burnham Greenway,41.712749,-87.534814,1,52


In [14]:
test.status.value_counts()

status
0    941
1    475
Name: count, dtype: int64

In [15]:
tables = ["clean_311", "clean_bike_stations", "clean_bus_stations", "clean_crime", "clean_cta_stations"]

In [16]:
def read_in_files(tables):
    for table in tables:
        globals()[table] = pd.read_csv(f"../datasets/clean/{table}.csv")
        globals()[table] = determine_within(globals()[table])
        print(f'{table} is done')

In [17]:
#read_in_files(tables)

In [18]:
clean_bike_stations = pd.read_csv('../datasets/clean/clean_bike_stations.csv')
clean_bike_stations

,id,station_name,lat,long,status,area
0,1594046383808271024,Troy St & Jackson Blvd,41.877505,-87.704850,1,27
1,641,Central Park Ave & Bloomingdale Ave,41.914166,-87.716755,1,22
2,1683527931525155814,Public Rack - Cornell Ave & 87th Pl,41.736881,-87.583146,0,48
3,367,Racine Ave & 35th St,41.830689,-87.656211,0,60
4,1673852313397164648,Public Rack - Francisco Ave & Touhy Ave,42.011865,-87.701317,0,2
...,...,...,...,...,...,...
1411,20,Sheffield Ave & Kingsbury St,41.910522,-87.653106,0,8
1412,54,Ogden Ave & Chicago Ave,41.896362,-87.654061,0,24
1413,693,Baltimore Ave & 87th St,41.737336,-87.548515,1,46
1414,718,Ewing Ave & Burnham Greenway,41.712749,-87.534814,1,52


In [19]:
clean_bike_stations = determine_within(clean_bike_stations)
clean_bike_stations

,id,station_name,lat,long,status,area
0,1594046383808271024,Troy St & Jackson Blvd,41.877505,-87.704850,1,27
1,641,Central Park Ave & Bloomingdale Ave,41.914166,-87.716755,1,22
2,1683527931525155814,Public Rack - Cornell Ave & 87th Pl,41.736881,-87.583146,0,48
3,367,Racine Ave & 35th St,41.830689,-87.656211,0,60
4,1673852313397164648,Public Rack - Francisco Ave & Touhy Ave,42.011865,-87.701317,0,2
...,...,...,...,...,...,...
1411,20,Sheffield Ave & Kingsbury St,41.910522,-87.653106,0,8
1412,54,Ogden Ave & Chicago Ave,41.896362,-87.654061,0,24
1413,693,Baltimore Ave & 87th St,41.737336,-87.548515,1,46
1414,718,Ewing Ave & Burnham Greenway,41.712749,-87.534814,1,52


In [20]:
clean_bike_stations.to_csv('../datasets/clean/clean_bike_stations.csv', index=False)

In [21]:
for table in tables:
    globals()[table].to_csv(f'../datasets/clean/{table}.csv', index=False)
    print(f"Just finished saving the {table} table to csv.")

KeyError: 'clean_311'

In [ ]:
clean_police_stations = pd.read_csv('../datasets/clean/clean_police_stations.csv')
clean_police_stations

,id,lat,long
0,0,41.830702,-87.623395
1,18,41.903242,-87.643352
2,19,41.947400,-87.651512
3,20,41.979550,-87.692845
4,22,41.691435,-87.668520
5,24,41.999763,-87.671324
6,25,41.918609,-87.765574
7,1,41.858373,-87.627356
8,2,41.801811,-87.630560
9,3,41.766431,-87.605748


In [ ]:
clean_police_stations = determine_within(clean_police_stations)
clean_police_stations.to_csv('../datasets/clean/clean_police_stations.csv', index=False)
clean_police_stations

,id,lat,long,status,area
0,0,41.830702,-87.623395,True,35
1,18,41.903242,-87.643352,False,8
2,19,41.947400,-87.651512,False,6
3,20,41.979550,-87.692845,False,4
4,22,41.691435,-87.668520,False,75
5,24,41.999763,-87.671324,False,1
6,25,41.918609,-87.765574,False,19
7,1,41.858373,-87.627356,False,33
8,2,41.801811,-87.630560,True,37
9,3,41.766431,-87.605748,True,42


In [17]:
clean_lighting = pd.read_csv('../datasets/clean/clean_lighting.csv')
clean_lighting = determine_within(clean_lighting)
clean_lighting

,Unnamed: 0,date,id,lat,long,status,area
0,0,2010-07-06,10-01015837,41.730274,-87.659251,1,73
1,1,2010-07-07,10-01015837,41.730274,-87.659251,1,73
2,2,2010-07-08,10-01015837,41.730274,-87.659251,1,73
3,3,2010-07-09,10-01015837,41.730274,-87.659251,1,73
4,4,2010-07-10,10-01015837,41.730274,-87.659251,1,73
...,...,...,...,...,...,...,...
9118467,9118467,2019-02-05,18-03384721,41.728848,-87.637507,1,71
9118468,9118468,2019-02-06,18-03384721,41.728848,-87.637507,1,71
9118469,9118469,2019-02-07,18-03384721,41.728848,-87.637507,1,71
9118470,9118470,2019-02-08,18-03384721,41.728848,-87.637507,1,71


In [19]:
clean_lighting = clean_lighting[['date', 'id', 'lat', 'long', 'status', 'area']]

In [15]:
clean_vacant_buildings = pd.read_csv('../datasets/clean/clean_vacant_buildings.csv')
clean_vacant_buildings = determine_within(clean_vacant_buildings)
clean_vacant_buildings

,Unnamed: 0.1,Unnamed: 0,start_date,id,lat,long,end_date,duration,status,area
0,0,0,2017-10-11,17-06828959,41.753463,-87.657041,2023-10-19,2199 days,1,71
1,1,1,2017-10-11,17-06827096,41.774412,-87.611883,2023-10-19,2199 days,1,42
2,2,2,2017-10-11,17-06818719,41.702514,-87.630974,2023-10-19,2199 days,1,49
3,3,3,2017-10-11,17-06824436,41.789879,-87.643825,2023-10-19,2199 days,1,68
4,4,4,2017-10-11,17-06826684,41.721604,-87.632602,2023-10-19,2199 days,1,49
...,...,...,...,...,...,...,...,...,...,...
65114,65114,65114,2018-12-17,18-03384005,41.741764,-87.556361,2023-10-19,1767 days,1,46
65115,65115,65115,2018-12-17,18-03380438,41.744965,-87.657233,2023-10-19,1767 days,1,71
65116,65116,65116,2018-12-17,18-03379703,41.752659,-87.578451,2023-10-19,1767 days,1,43
65117,65117,65117,2018-12-17,18-03382499,41.894108,-87.726045,2023-10-19,1767 days,1,23


In [20]:
clean_lighting.to_csv('../datasets/clean/clean_lighting.csv', index=False)
#clean_vacant_buildings.to_csv('../datasets/clean/clean_vacant_buildings.csv', index=False)